In [43]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

In [44]:
import pandas as pd
import os

# Create a table (DataFrame) with sector IDs and names
sector_list = [
    {"id_sector": 1, "sector": "technology"},
    {"id_sector": 2, "sector": "communication-services"},
    {"id_sector": 3, "sector": "consumer-cyclical"},
    {"id_sector": 4, "sector": "consumer-defensive"},
    {"id_sector": 5, "sector": "energy"},
    {"id_sector": 6, "sector": "healthcare"},
    {"id_sector": 7, "sector": "industrials"},
    {"id_sector": 8, "sector": "real-estate"},
    {"id_sector": 9, "sector": "utilities"}
]

sectors_df = pd.DataFrame(sector_list)
sectors_df.to_csv("../data/sectors.csv", index=False)

In [58]:
sector_list = [
    "technology", "communication-services", "consumer-cyclical",
    "consumer-defensive", "energy", "healthcare",
    "industrials", "real-estate", "utilities"
]

# Create sector_id mapping
sector_id_map = {sector: idx + 1 for idx, sector in enumerate(sector_list)}

data = []

for sector_name in sector_list:
    try:
        sector = yf.Sector(sector_name)
        industries_df = sector.industries

        # Check if data exists
        if industries_df is not None and not industries_df.empty:
            for key, row in industries_df.iterrows():
                data.append({
                    "sector_id": sector_id_map[sector_name],
                    "sector": sector_name,
                    "key": key,
                    "name": row["name"],
                    "symbol": row["symbol"],
                    "market_weight": row["market weight"]
                })
        else:
            print(f"[!] No data for sector: {sector_name}")
    except Exception as e:
        print(f"[!] Error retrieving data for sector '{sector_name}': {e}")

# Create DataFrame
df = pd.DataFrame(data)

# Add auto-incrementing ID for each key
df.insert(0, "id_key", range(1, len(df) + 1))

# Show a few rows
print(df.head())

# Save file if needed
df.to_csv("../data/all_sectors_data.csv", index=False)

   id_key  sector_id      sector                              key  \
0       1          1  technology                   semiconductors   
1       2          1  technology          software-infrastructure   
2       3          1  technology             consumer-electronics   
3       4          1  technology             software-application   
4       5          1  technology  information-technology-services   

                              name       symbol  market_weight  
0                   Semiconductors  ^YH31130020       0.311612  
1        Software - Infrastructure  ^YH31110030       0.282199  
2             Consumer Electronics  ^YH31120030       0.145690  
3           Software - Application  ^YH31110020       0.124728  
4  Information Technology Services  ^YH31110010       0.042663  


In [ ]:
# === STEP 1: FILTER STOCK LIST BY INDUSTRY ===
industry_name = "software-infrastructure"
industry = yf.Industry(industry_name)

# You can choose: top_growth_companies or top_performing_companies
industry_df = industry.top_growth_companies

# Get up to the first 10 tickers to avoid rate limits
tickers = industry_df.index.tolist()[:10]

# === STEP 2: GET PRICE DATA ===
def get_price_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="2d")
        if len(hist) < 2:
            return None

        last_price = hist['Close'][-1]
        prev_price = hist['Close'][-2]
        price_change = last_price - prev_price
        pct_change = (price_change / prev_price) * 100

        max_price = hist['High'].max()
        low_price = hist['Low'].min()

        return {
            "Stock Codes": ticker,
            "Last Price": round(last_price, 2),
            "Previous Day Price": round(prev_price, 2),
            "Price Change": round(price_change, 2),
            "% Change": f"{pct_change:.1f}%",
            "Max Price": round(max_price, 2),
            "Low Price": round(low_price, 2)
        }
    except:
        return None

data = [get_price_data(tk) for tk in tickers]
data = [d for d in data if d]
df = pd.DataFrame(data)

# === STEP 3: DISPLAY TABLE ===
fig = go.Figure(data=[go.Table(
    header=dict(
        values=list(df.columns),
        fill_color='darkblue',
        font=dict(color='white', size=12),
        align='center'
    ),
    cells=dict(
        values=[df[col] for col in df.columns],
        fill_color=[
            ['#d4edda' if isinstance(v, str) and '-' not in v and '%' in v
             else '#f8d7da' if isinstance(v, str) and '-' in v and '%' in v
             else 'white'] * len(df) for v in df.T.values
        ],
        align='center'
    ))
])

fig.update_layout(width=1000, height=400)
fig.show()


C:\Users\HP\AppData\Local\Temp\ipykernel_8296\2043910567.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  last_price = hist['Close'][-1]
C:\Users\HP\AppData\Local\Temp\ipykernel_8296\2043910567.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prev_price = hist['Close'][-2]
C:\Users\HP\AppData\Local\Temp\ipykernel_8296\2043910567.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  last_price = hist['Close'][-1]
C:\Users\HP\A

In [ ]:
# === STEP 1: FILTER STOCK LIST BY INDUSTRY ===
industry_name = "software-infrastructure"
industry = yf.Industry(industry_name)

industry_df = industry.top_growth_companies
tickers = industry_df.index.tolist()[:10]

# === STEP 2: GET PRICE DATA ===
def get_price_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="2d")
        if len(hist) < 2:
            return None

        last_price = hist['Close'][-1]
        prev_price = hist['Close'][-2]
        price_change = last_price - prev_price
        pct_change = (price_change / prev_price) * 100

        max_price = hist['High'].max()
        low_price = hist['Low'].min()

        return {
            "Stock Codes": ticker,
            "Last Price": round(last_price, 2),
            "Previous Day Price": round(prev_price, 2),
            "Price Change": round(price_change, 2),
            "% Change": f"{pct_change:.1f}%",
            "Max Price": round(max_price, 2),
            "Low Price": round(low_price, 2)
        }
    except:
        return None

data = [get_price_data(tk) for tk in tickers]
data = [d for d in data if d]
df = pd.DataFrame(data)

# === STEP 3: EXPORT TO CSV ===
df.to_csv("top_growth_stocks.csv", index=False)
print("✅ Đã lưu file: top_growth_stocks.csv")

C:\Users\HP\AppData\Local\Temp\ipykernel_8296\1990643687.py:16: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\HP\AppData\Local\Temp\ipykernel_8296\1990643687.py:17: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\HP\AppData\Local\Temp\ipykernel_8296\1990643687.py:16: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\HP\AppData\Local\Temp\ipykernel_8296\1990643687.py:17: FutureWarning:

Series.__getitem__ treatin

✅ Đã lưu file: top_growth_stocks.csv


C:\Users\HP\AppData\Local\Temp\ipykernel_8296\1990643687.py:16: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\HP\AppData\Local\Temp\ipykernel_8296\1990643687.py:17: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\HP\AppData\Local\Temp\ipykernel_8296\1990643687.py:16: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\HP\AppData\Local\Temp\ipykernel_8296\1990643687.py:17: FutureWarning:

Series.__getitem__ treatin

In [ ]:
ticker = 'AAPL'
data = yf.download(ticker, start='2018-01-01', end='2021-01-01', auto_adjust=False)

# Chọn cột giá điều chỉnh nếu có
price_col = 'Adj Close' if 'Adj Close' in data.columns else 'Close'

# Tính trung bình động
data['30D MA'] = data[price_col].rolling(window=30).mean()
data['90D MA'] = data[price_col].rolling(window=90).mean()

# Tính volume cao nhất đến thời điểm hiện tại (cumulative max)
data['Prev Volume High'] = data['Volume'].cummax()

# Chỉ giữ các cột cần thiết
data_selected = data[[price_col, '30D MA', '90D MA', 'Volume', 'Prev Volume High']]

# Xem trước dữ liệu
print(data_selected.tail())

[*********************100%***********************]  1 of 1 completed

Price        Adj Close      30D MA      90D MA     Volume Prev Volume High
Ticker            AAPL                               AAPL                 
Date                                                                      
2020-12-24  128.736969  119.508075  115.768225   54930100        426510000
2020-12-28  133.341309  120.076466  115.997812  124486200        426510000
2020-12-29  131.565933  120.584053  116.179892  121047300        426510000
2020-12-30  130.444092  121.020427  116.283557   96452100        426510000
2020-12-31  129.439362  121.452900  116.359964   99116600        426510000


In [7]:
data_selected.to_csv('aapl_processed.csv')

In [ ]:
# The following function, STOCK, retrieves historical price data for a list of stock tickers using the yfinance library.
# For each ticker in the provided list, it downloads the full available historical data (period="max") and stores it in a dictionary,
# where the keys are the ticker symbols and the values are pandas DataFrames containing the historical data for each stock.
# Example usage is shown below, where the function is called with a list of tickers, and the resulting dictionary is displayed.

def STOCK(tickers):
    """
    Retrieve historical price data for a list of stock tickers.

    Parameters:
        tickers (list of str): List of stock ticker symbols.

    Returns:
        dict: A dictionary where each key is a ticker symbol and each value is a pandas DataFrame
              containing the historical price data for that ticker.
    """
    data = {}
    for ticker in tickers:
        data[ticker] = yf.Ticker(ticker).history(period="max")
    return data

# Example usage:
tickers = ["AAPL", "MSFT", "GOOGL"]  # List of example stock tickers
df_dict = STOCK(tickers)
df_dict  # Dictionary: {ticker: DataFrame of historical data}

{'AAPL':                                  Open        High         Low       Close  \
 Date                                                                        
 1980-12-12 00:00:00-05:00    0.098597    0.099025    0.098597    0.098597   
 1980-12-15 00:00:00-05:00    0.093881    0.093881    0.093453    0.093453   
 1980-12-16 00:00:00-05:00    0.087022    0.087022    0.086594    0.086594   
 1980-12-17 00:00:00-05:00    0.088737    0.089165    0.088737    0.088737   
 1980-12-18 00:00:00-05:00    0.091310    0.091738    0.091310    0.091310   
 ...                               ...         ...         ...         ...   
 2025-06-23 00:00:00-04:00  201.630005  202.300003  198.960007  201.500000   
 2025-06-24 00:00:00-04:00  202.589996  203.440002  200.199997  200.300003   
 2025-06-25 00:00:00-04:00  201.449997  203.669998  200.619995  201.559998   
 2025-06-26 00:00:00-04:00  201.429993  202.639999  199.460007  201.000000   
 2025-06-27 00:00:00-04:00  201.889999  203.220001  200.

In [ ]:
# Get a DataFrame with all securities matching the query "AAPL" (up to 100 results)
all = yf.Lookup("AAPL").get_all(count=100)
all

,exchange,industryLink,industryName,quoteType,rank,regularMarketChange,regularMarketPercentChange,regularMarketPrice,shortName
symbol,,,,,,,,,
AAPL,NMS,https://finance.yahoo.com/sector/technology,Technology,equity,34658.0,0.080002,0.039802,201.080002,Apple Inc.
AAPL.MX,MEX,https://finance.yahoo.com/sector/technology,Technology,equity,20122.0,-3.790039,-0.099896,3790.199951,APPLE INC
AAPU,NGM,NaN,NaN,etf,20017.0,0.020001,0.094879,21.100000,Direxion Daily AAPL Bull 2X Sha
AAPL.NE,NEO,https://finance.yahoo.com/sector/technology,Technology,equity,20006.0,-0.080000,-0.274442,29.070000,APPLE CDR (CAD HEDGED)
AAPB,NGM,NaN,NaN,etf,20005.0,0.096001,0.497052,19.410000,GraniteShares 2x Long AAPL Dail
AAPL.TO,TOR,https://finance.yahoo.com/sector/technology,Technology,equity,20004.0,0.760000,2.677943,29.139999,APPLE CDR (CAD HEDGED)
AAPS.L,LSE,NaN,NaN,etf,20002.0,-0.021500,-2.103717,1.000500,LEVERAGE SHARES PUBLIC LIMITED
APLY,PCX,NaN,NaN,etf,20002.0,0.040000,0.314960,12.740000,Tidal ETF Trust II YieldMax AAP
APLY.NE,NEO,NaN,NaN,etf,20002.0,-0.020000,-0.082476,24.230000,APPLE (AAPL) YIELD SHARES PURPO


In [ ]:
stock = yf.Lookup("AAPL").get_stock(count=100)
stock

,exchange,industryLink,industryName,quoteType,rank,regularMarketChange,regularMarketPercentChange,regularMarketPrice,shortName
symbol,,,,,,,,,
AAPL,NMS,https://finance.yahoo.com/sector/technology,Technology,equity,34658.0,0.080002,0.039802,201.080002,Apple Inc.
AAPL.MX,MEX,https://finance.yahoo.com/sector/technology,Technology,equity,20122.0,-3.790039,-0.099896,3790.199951,APPLE INC
AAPL.NE,NEO,https://finance.yahoo.com/sector/technology,Technology,equity,20006.0,-0.080000,-0.274442,29.070000,APPLE CDR (CAD HEDGED)
AAPL.TO,TOR,https://finance.yahoo.com/sector/technology,Technology,equity,20004.0,0.760000,2.677943,29.139999,APPLE CDR (CAD HEDGED)
AAPLUSTRAD.BO,BSE,https://finance.yahoo.com/sector/industrials,Industrials,equity,20002.0,0.000000,0.000000,0.740000,AA Plus Tradelink Limited
AAPL.BA,BUE,https://finance.yahoo.com/sector/technology,Technology,equity,20002.0,-50.000000,-0.412371,12075.000000,APPLE INC CEDEAR(REPR 1/20 SHR)
AAPY.DE,GER,NaN,NaN,equity,20001.0,0.044000,0.617589,7.168500,Leverage Shares PLC E
XS246371164.DU,DUS,NaN,NaN,equity,20001.0,-0.009995,-0.010043,99.510002,NaN
AAPLC.BA,BUE,NaN,NaN,equity,20001.0,0.050000,0.497514,10.100000,APPLE INC CEDEAR(REPR 1/10 SHR)


In [ ]:


tech = yf.Sector('technology')
software = yf.Industry('software-infrastructure')

# Common information
tech.key
tech.name
tech.symbol
tech.ticker
tech.overview
tech.top_companies
tech.research_reports

# Sector information
tech.top_etfs
tech.top_mutual_funds
tech.industries

# Industry information
software.sector_key
software.sector_name
software.top_performing_companies
software.top_growth_companies

,name,ytd return,growth estimate
symbol,,,
CFLT,"Confluent, Inc.",-0.1248,5.500000
AVDX,"AvidXchange Holdings, Inc.",-0.0532,3.000000
OSPN,OneSpan Inc.,-0.0933,2.000000
GB,Global Blue Group Holding AG,0.0794,1.809818
IOT,Samsara Inc.,-0.0936,1.428571
